# 5.3 Categorical Features in Regression Models

So far, we have fit linear and $k$-nearest neighbors regression models to data where all of the features are quantitative. But what if all or some of the features are categorical? In theory, the solution is simple: we simply transform the categorical variables into quantitative variables using dummy (i.e., one-hot) encoding, following the process in Chapter 3. However, in practice, some care is needed to ensure that the categorical variables are transformed in a consistent way between the training and the test data.

In [1]:
import pandas as pd
df_housing = pd.read_csv("http://dlsun.github.io/pods/data/AmesHousing.txt", sep="\t")
df_housing.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


## One Categorical Feature

Let's develop some intuition about the predictions that a regression model will make when there is a single categorical feature. First, suppose we train a linear regression model to predict house price from the neighborhood the house is in.

In [2]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression

X = df_housing[["Neighborhood"]] # need 2D array for sklearn
y = df_housing["SalePrice"]

enc = OneHotEncoder()
X_dummies = enc.fit_transform(X)

model = LinearRegression()
model.fit(X_dummies, y)

LinearRegression()

A regression model with just a single feature, **Neighborhood**, will predict the same price for all houses in the same neighborhood. What is that predicted value? We can obtain it by applying the `OneHotEncoder` to a list of the unique neighborhoods in the data set and passing this to `model.predict()`.

One way to obtain a list of the unique neighborhoods is inside the encoder itself, under the attribute `.categories_`. We convert this to a 2D-array to be compatible with scikit-learn.

In [3]:
X_test = pd.Series(enc.categories_[0], name="Neighborhood").to_frame()
X_test

,Neighborhood
0,Blmngtn
1,Blueste
2,BrDale
3,BrkSide
4,ClearCr
5,CollgCr
6,Crawfor
7,Edwards
8,Gilbert
9,Greens


In [4]:
model.predict(enc.transform(X_test))

array([196661.6788359 , 143590.00024614, 105608.33360386, 124756.25045141,
       208662.09119794, 201803.43315691, 207550.83538449, 130843.3831155 ,
       190646.57663259, 193531.25024363, 280000.0002375 , 103752.90362624,
       137000.00023673,  95756.48676823, 162226.63205458, 145097.34964226,
       140710.86982888, 188406.9089579 , 330319.12710415, 322018.26594986,
       123991.8872417 , 135071.93780061, 136751.15302894, 184070.18452755,
       229707.32539161, 324229.19638331, 246599.54201249, 248314.58359813])

It is a bit hard to tell which prediction corresponds to which neighborhood. Let's put these numbers into a `Series`, indexed by the neighborhood.

In [5]:
pd.Series(
    model.predict(enc.transform(X_test)),
    index=X_test["Neighborhood"]
)

Neighborhood
Blmngtn    196661.678836
Blueste    143590.000246
BrDale     105608.333604
BrkSide    124756.250451
ClearCr    208662.091198
CollgCr    201803.433157
Crawfor    207550.835384
Edwards    130843.383116
Gilbert    190646.576633
Greens     193531.250244
GrnHill    280000.000237
IDOTRR     103752.903626
Landmrk    137000.000237
MeadowV     95756.486768
Mitchel    162226.632055
NAmes      145097.349642
NPkVill    140710.869829
NWAmes     188406.908958
NoRidge    330319.127104
NridgHt    322018.265950
OldTown    123991.887242
SWISU      135071.937801
Sawyer     136751.153029
SawyerW    184070.184528
Somerst    229707.325392
StoneBr    324229.196383
Timber     246599.542012
Veenker    248314.583598
dtype: float64

Could we have obtained these predictions some other way, without going through the trouble of fitting a linear regression model? Intuitively, if all we knew about a house was the neighborhood it was in, we would predict the average price of houses in that neighborhood.

In [6]:
df_housing.groupby("Neighborhood")["SalePrice"].mean()

Neighborhood
Blmngtn    196661.678571
Blueste    143590.000000
BrDale     105608.333333
BrkSide    124756.250000
ClearCr    208662.090909
CollgCr    201803.434457
Crawfor    207550.834951
Edwards    130843.381443
Gilbert    190646.575758
Greens     193531.250000
GrnHill    280000.000000
IDOTRR     103752.903226
Landmrk    137000.000000
MeadowV     95756.486486
Mitchel    162226.631579
NAmes      145097.349887
NPkVill    140710.869565
NWAmes     188406.908397
NoRidge    330319.126761
NridgHt    322018.265060
OldTown    123991.891213
SWISU      135071.937500
Sawyer     136751.152318
SawyerW    184070.184000
Somerst    229707.324176
StoneBr    324229.196078
Timber     246599.541667
Veenker    248314.583333
Name: SalePrice, dtype: float64

These numbers match the predictions from our linear regression model exactly. Linear regression simply predicts the average price in each neighborhood. 

To see this mathematically, recall that linear regression minimizes the total squared distance between the observed price and the predicted price:

$$ \text{sum of } (\text{price} - \widehat{\text{price}})^2. $$

After we expand the **Neighborhood** column into 28 dummy variables (e.g., $I\{ \text{Blmngtn} \}$, $I\{ \text{Blueste} \}$, etc.), one for each neighborhood, we can write the predicted price in the linear regression model as 

$$ \widehat{\text{price}} = c_1 I\{ \text{Blmngtn} \} + c_2 I\{ \text{Blueste} \} + \ldots + c_{28} I\{ \text{Veenker} \}. $$

(For simplicity, we have omitted the intercept term $b$.)

Now, consider a house in Bloomington Heights, for which $I\{ \text{Blmngtn} \} = 1$ and all of the other dummy variables $I\{ \text{Blueste} \} = \ldots = I\{ \text{Veenker} \} = 0$. Then, $\widehat{\text{price}}$ for a house in Bloomington Heights is $c_1$. Likewise, $\widehat{\text{price}}$ for a house in Bluestem is $c_2$. And so forth.

Now, we can reframe linear regression as learning the values $c_1, c_2, \ldots, c_{28}$ that minimize

$$ \text{sum of } (\text{price} - \widehat{\text{price}})^2 = \underbrace{\text{sum of } (\text{price} - c_1)^2}_{\text{over houses in Blmngtn}} + \underbrace{\text{sum of } (\text{price} - c_2)^2}_{\text{over houses in Blueste}} + \ldots + \underbrace{\text{sum of } (\text{price} - c_{28})^2}_{\text{over houses in Veenker}}. $$ 

We saw in Chapter 3 that the value of $c$ that mimimizes the $\text{sum of } (\text{price} - c)^2$ is the mean of the prices. So $\hat c_1$ will be the average price of houses in Bloomington Heights, $\hat c_2$ the average price of houses in Bluestem, and so on. Since $\hat c_1, \hat c_2, \ldots, \hat c_{28}$ are also the predicted values for each neighborhood, this shows that linear regression will predict the average label in each category when there is only one categorical variable in the model.

Exercise 1 in this lesson asks you to investigate what $k$-nearest neighbors regression does in the same situation.

## Mixing Quantitative and Categorical Features

In general, we want to fit machine learning models that use a mix of both categorical and quantitative features. In this situation, we will want to apply the `OneHotEncoder` to only the categorical features. Scikit-learn provides a `ColumnTransformer` that allows us to selectively apply transformations to certain columns.

For example, suppose we want to fit a $k$-nearest neighbors model to predict house price from quantitative features (square footage, number of bedrooms, number of full bathrooms) and categorical features (neighborhood, building type). We can use a `ColumnTransformer` to standardize the quantitative features and one-hot encode the categorical features.

In [7]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

ct = make_column_transformer(
    (StandardScaler(), ["Gr Liv Area", "Bedroom AbvGr", "Full Bath"]),
    (OneHotEncoder(), ["Neighborhood", "Bldg Type"]),
    remainder="drop"  # all other columns in X will be dropped.
)
ct

ColumnTransformer(transformers=[('standardscaler', StandardScaler(),
                                 ['Gr Liv Area', 'Bedroom AbvGr', 'Full Bath']),
                                ('onehotencoder', OneHotEncoder(),
                                 ['Neighborhood', 'Bldg Type'])])

Next, we integrate this `ColumnTransformer` into a pipeline (refer to the previous lesson) with the `KNeighborsRegressor` model.

In [8]:
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsRegressor

pipeline = make_pipeline(
    ct,
    KNeighborsRegressor(n_neighbors=10)
)

pipeline.fit(X=df_housing[["Gr Liv Area", "Bedroom AbvGr", "Full Bath",
                           "Neighborhood", "Bldg Type"]], 
             y=df_housing["SalePrice"])

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['Gr Liv Area',
                                                   'Bedroom AbvGr',
                                                   'Full Bath']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['Neighborhood',
                                                   'Bldg Type'])])),
                ('kneighborsregressor', KNeighborsRegressor(n_neighbors=10))])

Now, if we wanted to use this model to predict the price of a 3BR/2BA, 1700 sqft single-family house in Bloomington Heights, we could create a `Series` with this information, and call `pipeline.predict()` on a 2D-array with this single row.

In [9]:
x_test = pd.Series()
x_test["Gr Liv Area"] = 1700
x_test["Bedroom AbvGr"] = 3
x_test["Full Bath"] = 2
x_test["Neighborhood"] = "Blmngtn"
x_test["Bldg Type"] = "1Fam"

pipeline.predict(X=pd.DataFrame([x_test]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


array([251550.])

So this house is predicted to cost $251,550.

## Exercises

1\. Using the Ames data set, build a $10$-nearest neighbors model to predict house price using **Neighborhood** as the only feature. How do the predictions compare with just using the mean house price of each neighborhood? If there are any discrepancies, can you explain why?

In [10]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsRegressor

X = df_housing[["Neighborhood"]] # need 2D array for sklearn
y = df_housing["SalePrice"]

enc = OneHotEncoder()
X_dummies = enc.fit_transform(X)

model = KNeighborsRegressor()
model.fit(X_dummies, y)

KNeighborsRegressor()

In [11]:
pd.Series(
    model.predict(enc.transform(X_test)),
    index=X_test["Neighborhood"]
)

Neighborhood
Blmngtn    194722.2
Blueste    156200.0
BrDale     113300.0
BrkSide    101480.0
ClearCr    223900.0
CollgCr    169800.0
Crawfor    238900.0
Edwards    117120.0
Gilbert    206065.0
Greens     188150.0
GrnHill    188700.0
IDOTRR     122940.8
Landmrk    121300.0
MeadowV    105080.0
Mitchel    133100.0
NAmes      147850.0
NPkVill    142380.0
NWAmes     182080.0
NoRidge    293210.0
NridgHt    310660.0
OldTown    124580.0
SWISU      143100.0
Sawyer     141730.0
SawyerW    148080.0
Somerst    246100.0
StoneBr    281385.2
Timber     200800.0
Veenker    242400.0
dtype: float64

2\. In the example from the lesson, we standardized the quantitative features and one-hot encoded the categorical features in parallel. This means that the dummy variables were not standardized before being passed into the $10$-nearest neighbors model. How would you modify the pipeline so that *all* of the variables are standardized?

(_Hint:_ You may find the `remainder="passthrough"` option of `ColumnTransformer` helpful.)

In [12]:
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.preprocessing import StandardScaler, OneHotEncoder

ct = make_column_transformer(
    (OneHotEncoder(), ["Neighborhood", "Bldg Type"]),
    remainder="passthrough",
    sparse_threshold=0
)

pipeline = make_pipeline(
    ct,
    StandardScaler(),
    KNeighborsRegressor(n_neighbors=10)
)

X_train = df_housing[["Gr Liv Area", "Bedroom AbvGr", "Full Bath", "Neighborhood", "Bldg Type"]]
y_train = df_housing["SalePrice"]

pipeline.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough', sparse_threshold=0,
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['Neighborhood',
                                                   'Bldg Type'])])),
                ('standardscaler', StandardScaler()),
                ('kneighborsregressor', KNeighborsRegressor(n_neighbors=10))])

3\. Using the tips data set (http://dlsun.github.io/pods/data/tips.csv ), use a $5$-nearest neighbors model to predict how much a male diner will tip on a Sunday bill of \$40.00.

In [13]:
tips = pd.read_csv("http://dlsun.github.io/pods/data/tips.csv")
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [14]:
ct = make_column_transformer(
    (OneHotEncoder(), ["sex", "day"]),
    (StandardScaler(), ["total_bill", "size"]),
    remainder="drop"
)
ct

ColumnTransformer(transformers=[('onehotencoder', OneHotEncoder(),
                                 ['sex', 'day']),
                                ('standardscaler', StandardScaler(),
                                 ['total_bill', 'size'])])

In [15]:
pipeline = make_pipeline(
    ct,
    KNeighborsRegressor(n_neighbors=5)
)

pipeline.fit(X=tips[["total_bill", "size", "sex", "day"]],
             y=tips["tip"])

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['sex', 'day']),
                                                 ('standardscaler',
                                                  StandardScaler(),
                                                  ['total_bill', 'size'])])),
                ('kneighborsregressor', KNeighborsRegressor())])

In [16]:
x_test = pd.Series({
    "total_bill": 40,
    "sex": "Male",
    "day": "Sun",
    "size": 1
})
pipeline.predict(pd.DataFrame([x_test]))

array([2.802])